In [ ]:
import pandas as pd
from datetime import timedelta

In [ ]:
DEOP = pd.read_csv('../Data/DEOP/2023_DEOP_Interp.csv',
                    parse_dates=['DateTime'],
                    index_col=['DateTime']).astype(float)

Const = pd.read_csv('../Data/Analysis/Const/Const.csv',
                    index_col=['Parameter'])

In [ ]:
## Base operational load

### Electrolyser ###
P_elsb = Const.loc['P_elsbUnit']*Const.loc['N_el'] # Standby power of electrolyser (kW)

Water_el = Const.loc['W_elUnit']*Const.loc['N_el'] # water usage of electrolyser when running at full capacity (mL/h)

### Water Tank ###
P_wtsb = Const.loc['P_wtmsbUnit']*1e-3*Const.loc['N_wtm'] # standby power of water tank (kW)
P_wtmop = Const.loc['P_wtmopUnit']*1e-3*Const.loc['N_wtm'] # power usage of water tank when in operation (kW)
Water_wtm = Const.loc['W_wtmUnit']*1e3*Const.loc['N_wtm'] # water output of water tank when running at full capacity (mL/h)
E_wtm = P_wtmop/Water_wtm # Energy usage per unit water output (kWh/mL)
P_wtmeff = E_wtm*Water_el # Effective power usage of water tank to cover operation of electrolyser (kW)
P_wtmeffFrac = float('%.3g' % (P_wtmeff/Const.loc['P_elmax']).values[0]) # Fraction of maximum power usage of electrolyser used to output water needed.
print('P_wtmeffFrac = ',P_wtmeffFrac)

### Dryer ###
P_dsb = Const.loc['P_dsbUnit']*1e-3*Const.loc['N_d'] # standby power of dryer
P_dop = Const.loc['P_dopUnit']*1e-3*Const.loc['N_d'] # power usage of the dryer when in operation (kW)
H2_d = Const.loc['m_dUnit']*Const.loc['rho_spec']*Const.loc['N_d'] # Drying rate of hydrogen (kg/h)
E_d = P_dop/H2_d # Energy usage per unit hydrogen dried (kWh/kg)

### Refuelling System ###
E_rf = Const.loc['P_rfmax']/Const.loc['m_rfmax'] # Energy usage per unit hydrogen refuelling

### Battery ###
# The fans in one unit are rated 20-40W, so 20W is used to approximate the fan power.
P_battad = Const.loc['P_fanmin']*1e-3*(round(Const.loc['E_battcap']/Const.loc['E_battUnit'])) # Additional power used by battery (e.g. by fans) (kWh)
P_battadFrac = float('%.3g' % (P_battad/Const.loc['E_battcap']).values[0]) # Fraction of the battery energy storage capacity used to power additional battery operation
print('P_battadFrac = ',P_battadFrac)

### Combination ###
P_sb = P_elsb+P_wtsb+P_dsb # Total standby power usage
P_sbelFrac = float('%.3g' % (P_sb/Const.loc['P_elmax']).values[0]) # Fraction of maximum power usage of electrolyser used when in standby mode.
print('P_sbelFrac = ',P_sbelFrac)
E_H2 = float('%.3g' % (E_rf + E_d).values[0]) # Total energy usage per unit hydrogen produced (kWh/kg)
print('E_H2 = ',E_H2,' kWh/kg')

## Creating a function to model generation of hydrogen

In [ ]:
def HydrogenProduced(DataFrame,
                     eta_el=Const.loc['eta_el'].values[0],
                     eta_c=Const.loc['eta_c'].values[0],
                     eta_d=Const.loc['eta_d'].values[0],
                     P_elmax=Const.loc['P_elmax'].values[0],
                     LHV=Const.loc['LHV'].values[0],
                     P_sbFrac=P_sbelFrac,
                     E_H2=E_H2,
                     P_wtmeffFrac=P_wtmeffFrac,
                     P_ad=0,
                     E_battcap=Const.loc['E_battcap'].values[0],
                     P_battmaxE_battcap=(Const.loc['P_battmax'].values[0]/Const.loc['E_battcap'].values[0]),
                     E_battminFrac=Const.loc['E_battminFrac'].values[0],
                     E_battmaxFrac=Const.loc['E_battmaxFrac'].values[0],
                     P_battadFrac=P_battadFrac,
                     WaitTime=Const.loc['WaitTime'].values[0],
                     TimeRes=Const.loc['TimeRes'].values[0],
                     PowerInput = 'Excess',
                     Battery = True):
    """
    Function to calculate hydrogen produced by energy input, returning a modified dataframe

    Parameters
    ----------
    DataFrame : pd.DataFrame
        Where the energy data is stored
    eta_el : float
        Efficiency of the electrolyser.
        Defaults to 0.625.
    eta_c : float
        Efficiency of charging the battery.
        Defaults to 0.95.
    eta_d : float
        Efficiency of discharging the battery.
        Defaults to 0.95.
    P_elmax : float
        Maximum operating power of electrolyser (kW).
        Defaults to 25.
    LHV : float
        Lower heating value of hydrogen (kWh/kg).
        Defaults to 33.33.
    P_sbelFrac : float
        Fraction of P_elmax used during standby (when not producing hydrogen).
        Defaults to 0.0129.
    E_H2 : float
        Energy per unit hydrogen used for other processed (drying, refuelling etc.) (kWh/kg).
        Defaults to 0.161.
    P_wtmeffFrac : float
        Fraction of P_elmax used to generate water.
        Can include other processes only active during hydrogen generation.
        Defaults to 2.58e-5.
    P_ad : float
        Additional power used throughout (kW)
        such as by the campus (buildings, electric vehicles etc.)
        and the battery cooling system
        Defaults to 0.
    E_battcap : float
        Battery energy capacity (kWh).
        Defaults to 2000.
    P_battmaxE_battcap : float
        The ratio of maximum battery charge/discharge rate to battery energy capacity.
        Defaults to 0.5.
    E_battminFrac : float
        The fraction of E_battcap which is the minimum energy stored in the battery.
        Defaults to 0.1.
    E_battmaxFrac : float
        The fraction of E_battcap which is the maximum energy stored in the battery.
        Defaults to 0.9.
    P_battadFrac : float
        The fraction of E_battcap which is the additional power used continuously by the battery.
        This is for processes such as cooling the battery.
        Defaults to 2.31e-3.
    WaitTime : integer
        The time to wait until the state of the electrolyser may be changed (minutes).
        Defaults to 60.
    TimeRes : float
        The time resolution of the data (minutes).
        Defaults to 5.
    PowerInput : str
        'Excess' : uses power excess (wind plus solar minus load).
        'Total' : uses total power generation (wind plus solar).
        'Wind' : uses total power generation (wind) minus a constant load.
        'Solar' : uses total power generation (solar) minus a constant load.
        'WindExcess' : uses excess power generation (wind).
        'SolarExcess' : uses excess power generation (solar).
        Defaults to 'Excess'.
    Battery: boolean
        If True, a battery is modelled.
        If False, a system with no battery is modelled (equivalent to E_battmax = 0)
        Defaults to True.
    
    Returns
    -------
    data : pd.DataFrame
        The data from the simulation.
    """
    df = DataFrame.copy() # copy DataFrame to edit
    # Setting new columns
    df['E_el'] = None # energy of the electrolyser
    df['beta'] = None # state of electrolyser
    
    df['power-gencomb-ave'] = ((df['power-gen-wt-ave'] + df['power-gen-pv-ave'] - P_ad - P_battadFrac*E_battcap).clip(lower=0))
    df['power-excess-ave'] = (df['power-gencomb-ave'] - df['power-con-ave']).clip(lower=0)
    df['power-wt-excess-ave'] = (df['power-gen-wt-ave'] - df['power-con-ave'] - P_ad - P_battadFrac*E_battcap).clip(lower=0)
    df['power-pv-excess-ave'] = (df['power-gen-pv-ave'] - df['power-con-ave'] - P_ad - P_battadFrac*E_battcap).clip(lower=0)
    df['power-wt-constexcess-ave'] = (df['power-gen-wt-ave'] - P_ad - P_battadFrac*E_battcap).clip(lower=0)
    df['power-pv-constexcess-ave'] = (df['power-gen-pv-ave'] - P_ad - P_battadFrac*E_battcap).clip(lower=0)
    ### Setting constants ###
    if E_battcap == 0:
        Battery = False

    WaitTimeStamp = timedelta(minutes=WaitTime)
    TimeResStamp = timedelta(minutes=TimeRes)
    StartTime = df.index.min()
    delta_t = TimeResStamp/timedelta(hours=1) # to convert into kWh

    df['E_batt'] = None # energy stored in the battery
    df['gamma'] = None # state of battery (charge = +1, none = 0, discharge = -1)

    if Battery is True:
        E_battmin = E_battcap*E_battminFrac
        E_battmax = E_battcap*E_battmaxFrac
        P_battmax = E_battcap*P_battmaxE_battcap

    if PowerInput == 'Excess':
        Pin = 'power-excess-ave'
    elif PowerInput == 'Total':
        Pin = 'power-gencomb-ave'
    elif PowerInput == 'Wind':
        Pin = 'power-wt-constexcess-ave'
    elif PowerInput == 'Solar':
        Pin = 'power-pv-constexcess-ave'
    elif PowerInput == 'SolarExcess':
        Pin = 'power-pv-excess-ave'
    elif PowerInput == 'WindExcess':
        Pin = 'power-wt-excess-ave'

    # Additional power usage when not producing hydrogen.
    P_sb = P_elmax*P_sbFrac 

    # Additional power usage from water tank when producing hydrogen.
    P_wteff = P_elmax*P_wtmeffFrac
    
    # Additional energy usage attributed to production of hydrogen.
    # This includes running the water tank, drying the hydrogen and refuelling vehicles.
    P_H2ad = (eta_el*P_elmax*E_H2)/LHV + P_wteff

    ### Setting initial conditions ###
    df.at[StartTime,'E_el'] = 0 # no energy in electrolyser

    if Battery is True:
        df.at[StartTime,'E_batt'] = E_battmin # battery starts at minimum
        df.at[StartTime,'gamma'] = 0 # battery starts not charging/discharging (0)
    df.at[StartTime,'beta'] = 0 # electrolyer starts as off

    ### Modelling algorithm ###
    for index,row in df.iloc[1:].iterrows(): # starting at the second row
        NowMinusWait = index - WaitTimeStamp
        NowMinusWaitTime = max(NowMinusWait, StartTime)
        
        NowMinusWaitSlice = df[(df.index >= NowMinusWaitTime) & (df.index < index)]
        if Battery is True:
            E_battPrev = df.loc[index-TimeResStamp,'E_batt']

        # if the electrolyser has been off for an hour (or since the beginning) or if it was on previously, 
        # we can turn it on or keep it on
        if (NowMinusWaitSlice['beta'] == 0).all() or df.loc[index-TimeResStamp,'beta'] == 1:
            if df.at[index,Pin] >= P_elmax + P_H2ad: # is there enough green energy to power the electrolyser?
                df.at[index,'E_el'] = P_elmax*delta_t # power the electrolyser
                df.at[index,'beta'] = 1 # turn the electrolyser on
                
                if Battery is True:
                    # if the battery isn't full and won't be over full if charged
                    if (E_battPrev + df.at[index,Pin]-(P_elmax+P_H2ad)*delta_t < E_battmax) and (df.at[index,Pin]-(P_elmax+P_H2ad)*delta_t > 0):
                        df.at[index,'E_batt'] = E_battPrev + (df.at[index,Pin]-(P_elmax+P_H2ad))*delta_t*eta_c # charge the battery
                        df.at[index,'gamma'] = 1 # charge the battery
                    else:
                        df.at[index,'gamma'] = 0 # battery not changing charge
                        df.at[index,'E_batt'] = E_battPrev # battery not changing charge
            elif Battery is True:
                # can discharge it and have enough charge to power the electrolyser
                    # Is the energy left greater than the minimum charge state of the battery?
                    # Is the energy needed less than the maximum which the battery can output?
                if ((E_battPrev - (P_elmax + P_H2ad - df.at[index,Pin])*delta_t/eta_d) > E_battmin) and \
                    (df.at[index,Pin]*delta_t + eta_d*min(P_battmax*delta_t,E_battPrev)) >= (P_elmax+P_H2ad)*delta_t:
                    
                    df.at[index,'E_batt'] = E_battPrev - (P_elmax + P_H2ad - df.at[index,Pin])*delta_t*eta_d # discharge the battery
                    df.at[index,'gamma'] = -1 # discharge the battery
                    df.at[index,'E_el'] = P_elmax*delta_t # power the electrolyser
                    df.at[index,'beta'] = 1 # turn the electrolyser on
                
                # not possible to power electrolyser, even with help from battery
                else:
                    df.at[index,'beta'] = 0 # turn the electrolyser off
                    df.at[index,'E_el'] = 0 # electrolyser not producing any hydrogen
                    
                    # Can we charge the battery with the energy that can't be used?
                    if (E_battPrev + (df.at[index,Pin] - P_sb)*delta_t*eta_c < E_battmax) and (df.at[index,Pin] - P_sb > 0): 
                        df.at[index,'E_batt'] = E_battPrev + min((df.at[index,Pin] - P_sb),P_battmax)*eta_c*delta_t # charge the battery
                        df.at[index,'gamma'] = 1 # charge the battery
                    else:
                        df.at[index,'E_batt'] = E_battPrev # battery not changing charge
                        df.at[index,'gamma'] = 0 # battery not changing charge
            else:
                df.at[index,'beta'] = 0 # turn the electrolyser off
                df.at[index,'E_el'] = 0 # electrolyser not producing any hydrogen
        # can't turn the electrolyser on yet            
        else:
            df.at[index,'beta'] = 0 # keep the electrolyser off
            df.at[index,'E_el'] = 0 # electrolyser not producing any hydrogen
            if Battery is True:
                # can we charge the battery with the energy that can't be used?
                if (E_battPrev + (df.at[index,Pin] - P_sb)*delta_t*eta_c < E_battmax) and (df.at[index,Pin] - P_sb > 0):
                    df.at[index,'E_batt'] = E_battPrev + min((df.at[index,Pin] - P_sb),P_battmax)*eta_c*delta_t # charge the battery
                    df.at[index,'gamma'] = 1 # charge the battery
                else:
                    df.at[index,'E_batt'] = E_battPrev # battery not changing charge
                    df.at[index,'gamma'] = 0 # battery not changing charge

    ### Calculate hydrogen produced ###
    df['Hydrogen'] = (eta_el*df['E_el'])/LHV
    print(f'Done {str(P_elmax)}kW electrolyser, {str(E_battcap)}kWh battery')

    return df

In [ ]:
PowerList = [25,50,100,250,500,1000,1500,2000] # Electrolyser power in kW
BatteryList = [0,50,100,500,1000,2000,4000] # Battery capacities in kWh

In [ ]:
for Power in PowerList:
    df = HydrogenProduced(DEOP,P_elmax = Power)
    df_to_save = df.reset_index().rename(columns={'index': 'DateTime'})
    df_to_save.to_csv(f'../Data/Analysis/Simulations/Electrolyser/Battery/Excess/{Power}.csv',
                      index=False,columns=['DateTime','Hydrogen','E_el','beta','E_batt','gamma'])

In [ ]:
for Power in PowerList:
    df = HydrogenProduced(DEOP,P_elmax = Power,PowerInput='Solar',P_ad=100)
    df_to_save = df.reset_index().rename(columns={'index': 'DateTime'})
    df_to_save.to_csv(f'../Data/Analysis/Simulations/Electrolyser/Battery/Solar/{Power}.csv',
                      index=False,columns=['DateTime','Hydrogen','E_el','beta','E_batt','gamma'])

In [ ]:
for Power in PowerList:
    df = HydrogenProduced(DEOP,P_elmax = Power,PowerInput='Wind',P_ad=100)
    df_to_save = df.reset_index().rename(columns={'index': 'DateTime'})
    df_to_save.to_csv(f'../Data/Analysis/Simulations/Electrolyser/Battery/Wind/{Power}.csv',
                      index=False,columns=['DateTime','Hydrogen','E_el','beta','E_batt','gamma'])

In [ ]:
for Battery in BatteryList:
    df = HydrogenProduced(DEOP,E_battcap = Battery)
    df_to_save = df.reset_index().rename(columns={'index': 'DateTime'})
    df_to_save.to_csv(f'../Data/Analysis/Simulations/Battery/Excess/{Battery}.csv',
                      index=False,columns=['DateTime','Hydrogen','E_el','beta','E_batt','gamma'])

In [ ]:
for Battery in BatteryList:
    df = HydrogenProduced(DEOP,E_battcap = Battery,PowerInput='Solar',P_ad=100)
    df_to_save = df.reset_index().rename(columns={'index': 'DateTime'})
    df_to_save.to_csv(f'../Data/Analysis/Simulations/Battery/Solar/{Battery}.csv',
                      index=False,columns=['DateTime','Hydrogen','E_el','beta','E_batt','gamma'])

In [ ]:
for Battery in BatteryList:
    df = HydrogenProduced(DEOP,E_battcap = Battery,PowerInput='Wind',P_ad=100)
    df_to_save = df.reset_index().rename(columns={'index': 'DateTime'})
    df_to_save.to_csv(f'../Data/Analysis/Simulations/Battery/Wind/{Battery}.csv',
                      index=False,columns=['DateTime','Hydrogen','E_el','beta','E_batt','gamma'])